# 预训练模型的微调
Hans Cao, First: Dec 6, 2024; Update: Dec 6, 2024

**预训练模型的微调（Fine-tuning）** 是指将一个在大规模数据集（例如Wikipedia、新闻文章或特定领域数据）上经过预训练的通用模型（如BERT、GPT或FinBERT）调整为适应特定下游任务的过程。通过微调，可以优化模型参数，使其在目标应用中的表现更优，同时利用其在预训练阶段所获得的知识。

### Step 1: Load dataset

金融短语库Financial Phrase Bank）是一个用于金融情感分类的公共数据集，该数据集来自金融新闻。该数据集包含4840条来自英语金融新闻的句子，并根据情感进行分类。数据集按5至8名注释员的同意率进行划分。数据集地址： https://huggingface.co/datasets/takala/financial_phrasebank 这里选择了 "sentences_allagree" 配置，表示使用标注一致性最高的句子。

In [32]:
from datasets import load_dataset
# 选择 financial_phrasebank 数据集（"sentences_allagree" 配置）
fpb = load_dataset('imdb')
fpb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


该数据集包含个训练样本，标签存储在'label'里，文本存储在'sentence'里。


### Step 2: Tokenize the dataset

使用BERT WordPiece tokenizer对输入进行符号化（Tokenization）：

In [36]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(samples):
    return tokenizer(samples['text'], truncation=True)

tokenized_fpb = fpb.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Step 3: Data collator for padding

现在，文本已经符号化了，它们需要Hugging Face的Data.to_tf_dataset方法转换为TensorFlow数据集。

In [37]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

### Step 4: Prepare the dataset for training

In [45]:
train_data = tokenized_fpb['train'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'label'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

validation_data = tokenized_fpb['test'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'label'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

### Step 5: Initialize & Compile model


使用 distilbert-base-uncased 模型，该模型已在大规模数据集上预训练。这里使用 TFAutoModelForSequenceClassification 类加载DistilBERT，并指定任务的标签数量（例如情感分析任务的3个类别：正面、负面、中立）。

微调过程中，我们没有显式地修改模型架构，因为预训练模型已经为分类任务配置好了输出层。但如果需要添加额外的任务特定层（如线性分类器），可以通过模型的子类化或修改模型结构来实现。

采用Adam优化器并设置较低的学习率，这是微调预训练模型时常见的做法。

对于分类任务，使用了交叉熵损失函数（sparse_categorical_crossentropy），适用于标签是整数格式的分类任务。

In [47]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers.legacy import Adam  # 使用 legacy.Adam 以提升性能

# Load pre-trained DistilBERT model for sequence classification
model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

### Step 6: Train the model

现在准备微调。和往常一样在模型上调用fit

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate if validation accuracy stops improving
# lr_scheduler = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=2, min_lr=1e-6)

# history = model.fit(
#     train_data,
#     validation_data=None,
#     epochs=3,
#     callbacks=[lr_scheduler]
# )

hist = model.fit(train_data, validation_data=validation_data, epochs=3)

# Optional: Save the trained model
# model.save_pretrained('finbert_finetuned')   finbert_finetuned 保存了训练后的模型，你可以进一步在实际任务中使用它进行推断。

Epoch 1/3
 111/1563 [=>............................] - ETA: 2:09:35 - loss: 0.5653 - accuracy: 0.7044

本次训练是在MacOS系统上进行的训练（Chip：Apple M1 Max，Memory：32GB）上取得时长，供参考。同时在训练的过程中，观察accuracy的变化，来重新调整学习率的数值，并进行重新训练。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

# 获取训练和验证的准确率
acc = hist.history['accuracy']
epochs = range(1, len(acc) + 1)

# 绘制训练准确率曲线
plt.plot(epochs, acc, 'b-', label='Training Accuracy')

# 添加标题和标签
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## FinBERT

FinBERT采用与Devlin等人（2019）相同的微调架构和优化选择。具体来说，我们使用一个简单的线性层作为分类层，并采用softmax激活函数。损失函数使用交叉熵损失（cross-entropy loss）。通过在金融语料库（PhraseBank、FiQA、AnalystTone）上的预训练，FinBERT显著提升了金融情感分类任务的性能，在多个数据集上相比通用BERT模型均表现出明显的准确率优势。


### 微调的优势：
- **领域适配**：微调能使预训练模型适应特定领域的任务（如FinBERT适用于金融文本）。
- **减少数据需求**：相比从零开始训练，微调所需的标注数据更少。
- **性能提升**：在微调后，预训练模型通常能在特定任务中实现最先进的性能表现。


微调是将预训练模型应用于专业领域和任务的一种高效方法，既利用了模型的通用理解能力，又对其进行了定制化调整。

## 文献
- Jacob Devlin, Ming-Wei Chang, Kenton Lee, and
Kristina Toutanova. 2019. Bert: Pre-training of deep
bidirectional transformers for language understand-
ing. In Proceedings of NAACL, pages 4171–4186.
- Yi Yang Mark Christopher Siy UY Allen Huang, FinBERT: A Pretrained Language Model for Financial Communications, http://arxiv.org/abs/2006.08097v2